<a href="https://colab.research.google.com/github/gantapriyanka1325/DeepLearningLab/blob/main/program8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

8 Design and implement GRU model with tensor flow / keras and check accuracy

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder



In [2]:
# 1. LOAD NETFLIX DATASET

df = pd.read_csv("/content/netflix_titles.csv")   # your dataset


# Keep only necessary columns
df = df[['type', 'description']].dropna()

# Encode labels: Movie=0, TV Show=1
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['type'])

texts = df['description'].astype(str).tolist()
labels = df['label'].values



In [3]:
# 2. TEXT TOKENIZATION

vocab_size = 5000
max_len = 100

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=max_len, padding='post')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    padded, labels, test_size=0.2, random_state=42
)



In [4]:
# 3. BUILD GRU MODEL

model = Sequential([
    Embedding(vocab_size, 64, input_length=max_len),
    GRU(64, return_sequences=False),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Show summary
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
# 4. TRAIN MODEL

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=64
)

Epoch 1/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 14s 97ms/step - accuracy: 0.6993 - loss: 0.6244 - val_accuracy: 0.6890 - val_loss: 0.6199
Epoch 2/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 10s 88ms/step - accuracy: 0.6991 - loss: 0.6124 - val_accuracy: 0.6890 - val_loss: 0.6212
Epoch 3/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 9s 79ms/step - accuracy: 0.7014 - loss: 0.6109 - val_accuracy: 0.6890 - val_loss: 0.6203
Epoch 4/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 9s 83ms/step - accuracy: 0.7121 - loss: 0.6022 - val_accuracy: 0.6890 - val_loss: 0.6201
Epoch 5/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 10s 86ms/step - accuracy: 0.7000 - loss: 0.6125 - val_accuracy: 0.6890 - val_loss: 0.6204


In [6]:
# 5. ACCURACY & LOSS

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n Test Accuracy: {acc:.4f}")
print(f" Test Loss: {loss:.4f}")



 Test Accuracy: 0.6890
 Test Loss: 0.6204
